There are n cities connected by m flights. Each flight starts from city u and arrives at v with a price w.

Now given all the cities and flights, together with starting city src and the destination dst, your task is to find the cheapest price from src to dst with up to k stops. If there is no such route, output -1.

__Example 1:__
```
Input: 
n = 3, edges = [[0,1,100],[1,2,100],[0,2,500]]
src = 0, dst = 2, k = 1
Output: 200
```
Explanation: 
The graph looks like this:

The cheapest price from city 0 to city 2 with at most 1 stop costs 200, as marked red in the picture.

__Example 2:__
```
Input: 
n = 3, edges = [[0,1,100],[1,2,100],[0,2,500]]
src = 0, dst = 2, k = 0
Output: 500
Explanation: 
The graph looks like this:
```
The cheapest price from city 0 to city 2 with at most 0 stop costs 500, as marked blue in the picture. 

Constraints:

* The number of nodes n will be in range [1, 100], with nodes labeled from 0 to n - 1.
* The size of flights will be in range [0, n * (n - 1) / 2].
* The format of each flight will be (src, dst, price).
* The price of each flight will be in the range [1, 10000].
k is in the range of [0, n - 1].
* There will not be any duplicated flights or self cycles.

__Dijkstra Solution__ 

Time Complexity $\mathcal O(V\log V)$

Time Space $\mathcal O(V)$

* __E__, total of flights
* __V__, total of cities

In [50]:
public class Solution {
    public int findCheapestPrice(int n, int[][] flights, int src, int dst, int K) {
        Map<Integer, List<Flight>> graph = new HashMap<>();
        for (int i = 0; i < flights.length; ++i) {
            int[] item = flights[i];
            Flight flight = new Flight(item[0], item[1], item[2]);
            graph.computeIfAbsent(item[0], x -> new ArrayList<>())
                .add(flight);
        }
        PriorityQueue<Node> minHeap = 
            new PriorityQueue<>((x, y) -> x.cost - y.cost);
        minHeap.offer(new Node(src, 0, 0));
        Set<Node> seen = new HashSet<>();
        while (!minHeap.isEmpty()) {
            Node cur = minHeap.poll();
            if (cur.city == dst) return cur.cost;
            if (cur.stops == K + 1) continue;
            if (seen.contains(cur)) continue;
            seen.add(cur);
            if (!graph.containsKey(cur.city)) continue;
            for (Flight flight : graph.get(cur.city)) {
                minHeap.offer(
                    new Node(
                        flight.dst, 
                        cur.cost + flight.cost, 
                        cur.stops + 1));
            }
        }
        return -1;
    }
    private class Flight {
        int src, dst, cost;
        public Flight(int src, int dst, int cost) {
            this.src = src;
            this.dst = dst;
            this.cost = cost;
        }        
    }
    private class Node {
        int city, cost, stops;
        public Node(int city, int cost, int stops) {
            this.city = city;
            this.cost = cost;
            this.stops = stops;
        }
    }
}

In [51]:
int[][] flights = {{4,1,1},{1,2,3},{0,3,2},{0,4,10},{3,1,1},{1,4,3}};
new Solution().findCheapestPrice(5, flights, 2, 1, 1);

-1

In [52]:
int[][] flights = {{0,1,100},{1,2,100},{0,2,500}};
new Solution().findCheapestPrice(3, flights, 0, 2, 0);

500

In [53]:
int[][] flights = {{0,1,5},{1,2,5},{0,3,2},{3,1,2},{1,4,1},{4,2,1}};
new Solution().findCheapestPrice(5, flights, 0, 2, 2);

7

__DFS with Memoization Solution__ 

Time Complexity $\mathcal O((V+E)\log V)$

Time Space $\mathcal O(V)$

* __E__, total of flights
* __V__, total of cities

In [54]:
// without memoization
public class Solution {
    public int findCheapestPrice(int n, int[][] flights, int src, int dst, int K) {
        Map<Integer, List<Node>> graph = new HashMap<>();
        for (int[] flight : flights) {
            graph.computeIfAbsent(flight[0], x -> new ArrayList<>())
                .add(new Node(flight[0], flight[1], flight[2]));
        }
        return dfs(graph, src, dst, K);
    }
    private int dfs(Map<Integer, List<Node>> graph, int src, int dst, int k) {
        if (k < 0) return -1;
        if (!graph.containsKey(src)) return -1;
        int answer = Integer.MAX_VALUE;
        for (Node node : graph.get(src)) {
            if (node.dst == dst) {
                answer = Math.min(answer, node.cost);
                continue;
            }
            int t = dfs(graph, node.dst, dst, k - 1);
            if (t != -1) {
                answer = Math.min(answer, t + node.cost);
            }
        }
        if (answer == Integer.MAX_VALUE) return -1;
        return answer;
    }
    private class Node {
        int src, dst, cost;
        public Node(int src, int dst, int cost) {
            this.src = src;
            this.dst = dst;
            this.cost = cost;
        }
    }
}

In [55]:
int[][] flights = {{4,1,1},{1,2,3},{0,3,2},{0,4,10},{3,1,1},{1,4,3}};
new Solution().findCheapestPrice(5, flights, 2, 1, 1);

-1

In [56]:
int[][] flights = {{0,1,100},{1,2,100},{0,2,500}};
new Solution().findCheapestPrice(3, flights, 0, 2, 0);

500

In [57]:
int[][] flights = {{0,1,100},{1,2,100},{0,2,500}};
new Solution().findCheapestPrice(3, flights, 0, 2, 1);

200

In [58]:
int[][] flights = {{0,1,5},{1,2,5},{0,3,2},{3,1,2},{1,4,1},{4,2,1}};
new Solution().findCheapestPrice(5, flights, 0, 2, 2);

7

In [127]:
// with memoization
public class Solution {
    public int findCheapestPrice(int n, int[][] flights, int src, int dst, int K) {
        Map<CacheKey, Integer> cache = new HashMap<>();
        Map<Integer, List<Node>> graph = new HashMap<>();
        for (int[] flight : flights) {
            graph.computeIfAbsent(flight[0], x -> new ArrayList<>())
                .add(new Node(flight[0], flight[1], flight[2]));
        }
        return dfs(graph, src, dst, K, cache);
    }
    private int dfs(
        Map<Integer, List<Node>> graph, int src, int dst, int K, 
        Map<CacheKey, Integer> cache
    ) {
        if (K < 0) return -1;
        if (!graph.containsKey(src)) return -1;
        CacheKey cacheKey = new CacheKey(src, dst, K);
        if (cache.containsKey(cacheKey)) return cache.get(cacheKey);
        int answer = Integer.MAX_VALUE;
        for (Node node : graph.get(src)) {
            if (node.dst == dst) {
                answer = Math.min(answer, node.cost);
                continue;
            }
            int t = dfs(graph, node.dst, dst, K - 1, cache);
            if (t != -1) {
                answer = Math.min(answer, t + node.cost);
            }
        }
        if (answer == Integer.MAX_VALUE) return -1;
        cache.put(cacheKey, answer);
        return answer;
    }
    private class Node {
        int src, dst, cost;
        public Node(int src, int dst, int cost) {
            this.src = src;
            this.dst = dst;
            this.cost = cost;
        }
    }
    private class CacheKey {
        int src, dst, k;
        CacheKey(int src, int dst, int k) {
            this.src = src;
            this.dst = dst;
            this.k = k;
        }
        @Override
        public int hashCode() {
            int hash = 17;
            hash = hash * 31 + src;
            hash = hash * 31 + dst;
            hash = hash * 31 + k;
            return hash;
        }
        public boolean equals(Object that) {
            if (that == null) return false;
            if (!(that instanceof CacheKey)) return false;
            if (this == that) return true;
            return hashCode() == that.hashCode();
        }
    }
}

In [128]:
int[][] flights = {{4,1,1},{1,2,3},{0,3,2},{0,4,10},{3,1,1},{1,4,3}};
new Solution().findCheapestPrice(5, flights, 2, 1, 1);

-1

In [129]:
int[][] flights = {{0,1,100},{1,2,100},{0,2,500}};
new Solution().findCheapestPrice(3, flights, 0, 2, 0);

500

In [130]:
int[][] flights = {{0,1,100},{1,2,100},{0,2,500}};
new Solution().findCheapestPrice(3, flights, 0, 2, 1);

200

In [131]:
int[][] flights = {{0,1,5},{1,2,5},{0,3,2},{3,1,2},{1,4,1},{4,2,1}};
new Solution().findCheapestPrice(5, flights, 0, 2, 2);

7

In [223]:
// https://leetcode.com/problems/cheapest-flights-within-k-stops/solution/
public class Solution {
    public int findCheapestPrice(int n, int[][] flights, int src, int dst, int K) {
        long[][] dist = new long[2][n];
        
        Arrays.fill(dist[0], Integer.MAX_VALUE);
        Arrays.fill(dist[1], Integer.MAX_VALUE);
        
        dist[0][src] = 0;
        dist[1][src] = 0;
        int[] stops = new int[n];
        for (int j = 0; j <= K; ++j) {
            for (int i = 0; i < flights.length; ++i) {
                int u = flights[i][0];
                int v = flights[i][1];
                int cost = flights[i][2];
                long du = dist[1 - j&1][u];
                long dv = dist[j&1][v];
                if (du + cost < dv) {
                    dist[j&1][v] = du + cost;
                }
            }
        }   
        if (dist[K&1][dst] != Integer.MAX_VALUE) return (int) dist[K&1][dst];
        return -1;
    }
}

In [224]:
int[][] flights = {{0,1,5},{1,2,5},{0,3,2},{3,1,2},{1,4,1},{4,2,1}};
new Solution().findCheapestPrice(5, flights, 0, 2, 2);

7

In [225]:
int[][] flights = {{0,1,100},{1,2,100},{0,2,500}};
new Solution().findCheapestPrice(3, flights, 0, 2, 1);

200

In [226]:
int[][] flights = {{0,1,100},{1,2,100},{0,2,500}};
new Solution().findCheapestPrice(3, flights, 0, 2, 0);

500